### Multiclass Classification

In [2]:
from tables import *
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

#View training data
#train.shape (45324, 101)
test.index

Int64Index([45324, 45325, 45326, 45327, 45328, 45329, 45330, 45331, 45332,
            45333,
            ...
            53451, 53452, 53453, 53454, 53455, 53456, 53457, 53458, 53459,
            53460],
           dtype='int64', length=8137)

In [4]:
y_train = train['y']
x_train = train._drop_axis(['y'], axis=1)

#Switch to numpy
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(test)

#### Convert y to 1-hot for tensorflow

In [4]:
#5 classes: 0,1,2,3,4
y_train_hot = np.zeros((y_train.shape[0], 5))
y_train_hot[np.arange(y_train.shape[0]), y_train] = 1

#see 1-hot
#y_train_hot[0:5]

#### Model Parameters

In [5]:
learning_rate = 0.01
epochs = 20
batch_size = 64
disp_step = 2
logs_path = './tmp/tensorflow_logs' #for tensorboard

#### Placeholders, fed by training data

In [6]:
with tf.name_scope('DATA'):
    X = tf.placeholder(tf.float32, [None, 100], name='X')
    y = tf.placeholder(tf.float32, [None, 5], name='y')

#### Variables, to be learned

In [7]:
with tf.name_scope('VARIABLES'):
    W = tf.Variable(tf.zeros([100, 5]), name='weights')
    b = tf.Variable(tf.zeros([5]), name='biases')

#### Softmax the output layer

In [8]:
with tf.name_scope('SOFTMAX'): 
    pred = tf.nn.softmax(tf.matmul(X, W) + b, name='prediction')

#### Define Loss function: Cross entropy

In [9]:
with tf.name_scope('LOSS'):
    loss = tf.losses.softmax_cross_entropy(y, tf.log(pred))
    tf.summary.scalar('loss', loss)

#### Optimization

In [10]:
with tf.name_scope('OPTIMIZER'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

#### Initialize and run session

In [14]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    writer = tf.summary.FileWriter(logs_path, sess.graph)
    merged_summaries = tf.summary.merge_all()
    
    sess.run(init)
    
    #Train the model
    
    #Run number of epochs
    for epoch in range(epochs):
        avg_loss = 0
        total_batch = int(x_train.shape[0]/batch_size)
        
        #Loop over the batchs for each epoch
        for i in range(total_batch):
            batch_xs = x_train[(i*batch_size):(i+batch_size)+batch_size-1]
            batch_ys = y_train_hot[(i*batch_size):(i+batch_size)+batch_size-1]
            
            _, c = sess.run([optimizer, loss], feed_dict={X: batch_xs, y: batch_ys})
            
            if i % 25 == 0:
                s = sess.run(merged_summaries, feed_dict={X: batch_xs, y: batch_ys})
                writer.add_summary(s, epoch*total_batch + i)
                
            avg_loss += c / total_batch
            
        if(epoch+1) % disp_step == 0:
            print("Epoch: ", '%04d' % (epoch+1), "loss=", "{:.9f}".format(avg_loss))
            
    print('Optimization finished.')
    
    ### EVALUTATE
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    print('Accuracy: ', sess.run(accuracy, feed_dict={X: x_train, y: y_train_hot}))

Epoch:  0002 loss= 0.012311707
Epoch:  0004 loss= 0.025605847
Epoch:  0006 loss= 0.071245542
Epoch:  0008 loss= 0.054519955
Epoch:  0010 loss= 0.084187069
Epoch:  0012 loss= 0.044476904
Epoch:  0014 loss= 0.087728533
Epoch:  0016 loss= 0.055237377
Epoch:  0018 loss= 0.029304576
Epoch:  0020 loss= 0.035769186
Optimization finished.
Accuracy:  0.3850278


#### Evaluate

In [12]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
print('Accuracy: ', sess.run(accuracy, feed_dict={X: x_train, y: y_train_hot}))

RuntimeError: Attempted to use a closed Session.

## KERAS ATTEMPT

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l2

num_examples = x_train.shape[0]

#Define model
model = Sequential([Dense(70, input_shape=(100,)), #a bit counterintuitve
                   Activation('relu'),
                   Dense(30),
                   Activation('relu'),
                   Dense(50),
                   Activation('relu'),
                   Dense(20),
                   Activation('relu'),
                   Dense(5),
                   Activation('softmax'),
])

In [18]:
### Training
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Easy way to convert to one-hot
y_train_hot2 = keras.utils.to_categorical(y_train, num_classes=5)

model.fit(x_train, y_train_hot2, epochs=400)

Epoch 1/400
45324/45324 [==============================] - 2s 43us/step - loss: 1.1017 - acc: 0.5377
Epoch 2/400
45324/45324 [==============================] - 2s 47us/step - loss: 0.8424 - acc: 0.6683
Epoch 3/400
45324/45324 [==============================] - 2s 45us/step - loss: 0.7395 - acc: 0.7144: 0s - loss: 0.7400 - acc
Epoch 4/400
45324/45324 [==============================] - 2s 47us/step - loss: 0.6912 - acc: 0.7362
Epoch 5/400
45324/45324 [==============================] - 2s 47us/step - loss: 0.6561 - acc: 0.7510
Epoch 6/400
45324/45324 [==============================] - 3s 57us/step - loss: 0.6203 - acc: 0.7643
Epoch 7/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.5918 - acc: 0.7778
Epoch 8/400
45324/45324 [==============================] - 2s 55us/step - loss: 0.5714 - acc: 0.7861
Epoch 9/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.5449 - acc: 0.7951
Epoch 10/400
45324/45324 [==============================] - 2s 55u

45324/45324 [==============================] - 2s 55us/step - loss: 0.3037 - acc: 0.8910
Epoch 82/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.3028 - acc: 0.8903
Epoch 83/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.3020 - acc: 0.8910
Epoch 84/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.3010 - acc: 0.8897
Epoch 85/400
45324/45324 [==============================] - 2s 55us/step - loss: 0.3027 - acc: 0.8907
Epoch 86/400
45324/45324 [==============================] - 2s 55us/step - loss: 0.2962 - acc: 0.8930
Epoch 87/400
45324/45324 [==============================] - 2s 55us/step - loss: 0.3003 - acc: 0.8919
Epoch 88/400
45324/45324 [==============================] - 2s 53us/step - loss: 0.2961 - acc: 0.8928
Epoch 89/400
45324/45324 [==============================] - 2s 52us/step - loss: 0.2979 - acc: 0.8919
Epoch 90/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.2978 - acc: 

45324/45324 [==============================] - 3s 60us/step - loss: 0.2588 - acc: 0.9063
Epoch 161/400
45324/45324 [==============================] - 3s 58us/step - loss: 0.2586 - acc: 0.9067
Epoch 162/400
45324/45324 [==============================] - 3s 66us/step - loss: 0.2566 - acc: 0.9060
Epoch 163/400
45324/45324 [==============================] - 3s 62us/step - loss: 0.2547 - acc: 0.9076
Epoch 164/400
45324/45324 [==============================] - 3s 58us/step - loss: 0.2580 - acc: 0.9055
Epoch 165/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.2571 - acc: 0.9072
Epoch 166/400
45324/45324 [==============================] - 3s 62us/step - loss: 0.2570 - acc: 0.9077
Epoch 167/400
45324/45324 [==============================] - 3s 62us/step - loss: 0.2578 - acc: 0.9055
Epoch 168/400
45324/45324 [==============================] - 3s 56us/step - loss: 0.2550 - acc: 0.9089
Epoch 169/400
45324/45324 [==============================] - 2s 51us/step - loss: 0.255

45324/45324 [==============================] - 2s 54us/step - loss: 0.2370 - acc: 0.9140
Epoch 240/400
45324/45324 [==============================] - 2s 53us/step - loss: 0.2366 - acc: 0.9140
Epoch 241/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.2336 - acc: 0.9168
Epoch 242/400
45324/45324 [==============================] - 2s 53us/step - loss: 0.2335 - acc: 0.9150
Epoch 243/400
45324/45324 [==============================] - 2s 52us/step - loss: 0.2320 - acc: 0.9169
Epoch 244/400
45324/45324 [==============================] - 2s 52us/step - loss: 0.2326 - acc: 0.9164
Epoch 245/400
45324/45324 [==============================] - 2s 53us/step - loss: 0.2341 - acc: 0.9150
Epoch 246/400
45324/45324 [==============================] - 2s 52us/step - loss: 0.2333 - acc: 0.9158
Epoch 247/400
45324/45324 [==============================] - 2s 51us/step - loss: 0.2339 - acc: 0.9152
Epoch 248/400
45324/45324 [==============================] - 2s 53us/step - loss: 0.232

45324/45324 [==============================] - 2s 53us/step - loss: 0.2192 - acc: 0.9202
Epoch 319/400
45324/45324 [==============================] - 2s 53us/step - loss: 0.2201 - acc: 0.9190
Epoch 320/400
45324/45324 [==============================] - 2s 52us/step - loss: 0.2188 - acc: 0.9207
Epoch 321/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.2193 - acc: 0.9197
Epoch 322/400
45324/45324 [==============================] - 2s 52us/step - loss: 0.2166 - acc: 0.9204
Epoch 323/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.2187 - acc: 0.9215
Epoch 324/400
45324/45324 [==============================] - 2s 53us/step - loss: 0.2182 - acc: 0.9207
Epoch 325/400
45324/45324 [==============================] - 2s 51us/step - loss: 0.2174 - acc: 0.9209
Epoch 326/400
45324/45324 [==============================] - 2s 49us/step - loss: 0.2154 - acc: 0.9214
Epoch 327/400
45324/45324 [==============================] - 2s 49us/step - loss: 0.218

45324/45324 [==============================] - 2s 53us/step - loss: 0.2069 - acc: 0.9239
Epoch 398/400
45324/45324 [==============================] - 2s 52us/step - loss: 0.2068 - acc: 0.9253
Epoch 399/400
45324/45324 [==============================] - 2s 54us/step - loss: 0.2088 - acc: 0.9228
Epoch 400/400
45324/45324 [==============================] - 2s 51us/step - loss: 0.2085 - acc: 0.9254


In [19]:
pred_one_hot = model.predict(x_test)
type(pred_one_hot)

numpy.ndarray

In [20]:
#Convert predictions back from 1-hot
pred = []

for line in pred_one_hot:
    pred.append(np.argmax(line))

In [21]:
d = {'Id': test.index, 'y': pred}
out = pd.DataFrame(d)
out.to_csv('output.csv', index=False)